In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path_drive = '/content/drive/MyDrive/TESIS/'

df1 = pd.read_csv(path_drive + 'df1.csv')
df1_clean = pd.read_csv(path_drive + 'df1_clean.csv')
df2 = pd.read_csv(path_drive + 'df2.csv')
df2_letras = pd.read_csv(path_drive + 'df2_letras.csv')


print("Archivos cargados exitosamente desde Google Drive.")

Archivos cargados exitosamente desde Google Drive.


#DF2 CANCIONES SEGUN EMOCIONES

Vamos a filtrar nuestro conjunto de datos de acuerdo a las caracteristcas que rescatamos de nuestro análisis para cada trastorno.

In [ ]:
# Filtrar las canciones según las características especificadas
df2_emocion = df2_letras[
    # Primer conjunto de condiciones: acústica alta, danceabilidad baja, energía baja, géneros relajantes
    ((df2_letras['Acousticness'] > 0.6) &
     (df2_letras['Danceability'] < 0.2) &
     (df2_letras['Energy'] < 0.4) &
     (df2_letras['Género'].isin(['Classical & Orchestral', 'Instrumental & Relaxation', 'Video Game Music', 'Indie', 'R&B/Soul', 'Lo-fi & Chill', 'Folk'])))
    |
    # Segundo conjunto de condiciones: acústica baja, danceabilidad alta, energía alta, géneros energéticos
    ((df2_letras['Acousticness'] < 0.4) &
     (df2_letras['Danceability'] > 0.6) &
     (df2_letras['Energy'] > 0.7) &
     (df2_letras['Género'].isin(['K-Pop', 'Rock', 'Country', 'Pop'])))
]

# **Sistema de recomendación por similitud: letra y características musicales**

Este es un sistema de recomendación de música que sugiere canciones similares a una seleccionada por el usuario, basándose en el sentimiento de la letra (alegre o triste) y el ritmo de la canción (bailable o no bailable), utilizando análisis de sentimiento y características musicales como la energía y la valencia.

* Si seleccionas "alegria", el sistema selecciona canciones con sentimiento positivo según VADER y con un ritmo alto (alto Total Positivity Score).
* Si seleccionas "tristeza", el sistema selecciona canciones con sentimiento negativo según VADER y con un ritmo bajo (bajo Total Positivity Score).


En resumen, cuando un usuario selecciona una canción triste y luego solicita una recomendación alegre, el sistema basa su recomendación principalmente en la similitud musical (por ejemplo, la valencia, energía y capacidad de baile), independientemente del tipo de sentimiento emocional que la canción original tenga. La recomendación será de una canción alegre, pero que comparte las características musicales relevantes (ritmo, tono, etc.) con la canción triste seleccionada por el usuario.

Este enfoque asegura que el sistema puede sugerir canciones que son similares en términos de características musicales, incluso si son emocionalmente opuestas.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Función para recomendar una canción similar según el sentimiento y el ritmo
def recomendar_cancion(cancion_usuario, sentimiento, df):
    # Filtrar por sentimiento
    if sentimiento == 'alegria':
        df_filtrado = df[df['VADER_sentiment'] > 0]  # Sentimiento positivo
    else:
        df_filtrado = df[df['VADER_sentiment'] < 0]  # Sentimiento negativo

    # Filtrar por ritmo (usamos Total Positivity Score como criterio)
    if sentimiento == 'alegria':
        df_filtrado = df_filtrado[df_filtrado['Total Positivity Score'] > 0.5]  # Ritmo alto
    else:
        df_filtrado = df_filtrado[df_filtrado['Total Positivity Score'] < 0.5]  # Ritmo bajo

    # Convertir las canciones en vectores (puedes usar más características si lo deseas)
    canciones_vectors = df_filtrado[['Valence', 'Energy', 'Danceability']]

    # Convertir la canción seleccionada por el usuario a un vector
    cancion_vector = np.array([cancion_usuario['Valence'], cancion_usuario['Energy'], cancion_usuario['Danceability']]).reshape(1, -1)

    # Calcular la similitud coseno entre la canción seleccionada y las demás
    similitudes = cosine_similarity(cancion_vector, canciones_vectors)

    # Ordenar las canciones por similitud
    df_filtrado['similitud'] = similitudes[0]
    df_filtrado = df_filtrado.sort_values('similitud', ascending=False)

    # Filtrar para no incluir la canción seleccionada
    df_filtrado = df_filtrado[df_filtrado['Nombre'] != cancion_usuario['Nombre']]

    # Seleccionar las canciones más similares
    recomendacion = df_filtrado.head(3)

    # Mostrar los resultados con la similitud en el formato deseado
    canciones_recomendadas = []
    for _, row in recomendacion.iterrows():
        canciones_recomendadas.append({
            'artista': row['Artista'],
            'genero': row['Género'],
            'cancion_similar': row['Nombre'],
            'similitud': row['similitud']
        })

    return canciones_recomendadas

# Función principal para interactuar con el usuario
def obtener_recomendacion():
    # Solicitar al usuario el nombre de la canción de su interés
    while True:  # Usamos un bucle para permitir reintentos
        cancion_seleccionada = input("Por favor, ingrese el nombre de la canción que le interesa: ")

        # Verificar si la canción existe en el dataframe
        if cancion_seleccionada in df2_letras['Nombre'].values:
            cancion_usuario = df2_letras[df2_letras['Nombre'] == cancion_seleccionada].iloc[0]
            break  # Salir del bucle si la canción está en la base de datos
        else:
            print("La canción no se encuentra en el sistema. Intente nuevamente.")  # Mensaje de error

    # Solicitar al usuario el tipo de sentimiento que desea (alegria o tristeza)
    while True:
        sentimiento_usuario = input("¿Qué tipo de sentimiento desea que le transmita? (alegria/tristeza): ").lower()
        if sentimiento_usuario in ['alegria', 'tristeza']:
            break  # Salir del bucle si la entrada es válida
        else:
            print("Entrada no válida. Por favor ingrese 'alegria' o 'tristeza'.")  # Mensaje de error para entrada no válida

    # Llamar a la función de recomendación
    resultado = recomendar_cancion(cancion_usuario, sentimiento_usuario, df2_letras)

    # Mostrar las canciones recomendadas con la similitud en el formato solicitado
    print("\nCanciones recomendadas y sus similitudes:")
    for recomendacion in resultado:
        print(f"\nArtista: {recomendacion['artista']} ({recomendacion['genero']})")
        print(f"Canción similar: {recomendacion['cancion_similar']}")
        print(f"Similitud: {recomendacion['similitud']:.4f}")

# Llamar a la función principal para empezar la interacción con el usuario
obtener_recomendacion()


Por favor, ingrese el nombre de la canción que le interesa: Espresso
¿Qué tipo de sentimiento desea que le transmita? (alegria/tristeza): alegría
Entrada no válida. Por favor ingrese 'alegria' o 'tristeza'.
¿Qué tipo de sentimiento desea que le transmita? (alegria/tristeza): alegria

Canciones recomendadas y sus similitudes:

Artista: Lady Gaga (Pop)
Canción similar: Die With A Smile
Similitud: 1.0000

Artista: Bailey Zimmerman (nan)
Canción similar: Religiously
Similitud: 0.9997

Artista: Smash Mouth (Rock)
Canción similar: All Star
Similitud: 0.9996


# **Sistema de recomendación por similitud: artistas y canciones**

Este sistema sugiere artistas similares y sus canciones más parecidas basándose en el sentimiento de sus letras y las características musicales de sus canciones.

¿Cómo funciona?
- Selección del artista: El usuario ingresa el nombre de un artista, y el sistema verifica que esté en la base de datos.

- Tipos de recomendación:

Similitud por sentimiento (VADER): Se analiza el sentimiento de las canciones del artista seleccionado (positivo o negativo). Luego, el sistema recomienda artistas con sentimientos similares a los de ese artista.

1. Recomendación de artistas similares:

Se filtran los artistas con un sentimiento similar al del artista seleccionado, y se devuelven los tres artistas más similares.

2. Recomendación de canciones:

Para cada artista recomendado, el sistema encuentra canciones similares a las del artista seleccionado, comparando características musicales como valencia y energía.
Las canciones se ordenan por similitud (medida usando el cálculo de similitud coseno), y se evita recomendar canciones repetidas.

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Función para recomendar artistas similares
def recomendar_artistas(artista_usuario, df, tipo_recomendacion='vader'):
    # Filtrar para no incluir el propio artista
    df_filtrado = df[df['Artista'] != artista_usuario]

    if tipo_recomendacion == 'vader':
        # Filtrar por sentimiento (usamos el valor de VADER)
        vader_sentiment = df[df['Artista'] == artista_usuario]['VADER_sentiment'].iloc[0]
        if vader_sentiment > 0:
            df_filtrado = df_filtrado[df_filtrado['VADER_sentiment'] > 0]  # Similar en sentimiento positivo
        else:
            df_filtrado = df_filtrado[df_filtrado['VADER_sentiment'] < 0]  # Similar en sentimiento negativo

    # Seleccionar los artistas más similares (por defecto los 3 primeros)
    recomendacion_artistas = df_filtrado.head(3)  # Devolvemos los 3 artistas más similares
    return recomendacion_artistas


# Función para obtener canciones similares de los artistas recomendados
def obtener_canciones_similares(artista_usuario, artistas_recomendados, df):
    canciones_similares = []
    canciones_recomendadas = set()  # Para evitar canciones repetidas

    # Buscar canciones similares para cada uno de los artistas recomendados
    for idx, row in artistas_recomendados.iterrows():
        artista_recomendado = row['Artista']
        # Buscar las canciones de ese artista
        canciones = df[df['Artista'] == artista_recomendado]

        # Ordenar canciones por similitud
        similitudes = cosine_similarity(
            df[df['Artista'] == artista_usuario][['Valence', 'Energy']],
            canciones[['Valence', 'Energy']]
        )
        canciones['similitud'] = similitudes[0]

        # Seleccionar la canción más similar que no haya sido recomendada aún
        for i, cancion in canciones.iterrows():
            if cancion['Nombre'] not in canciones_recomendadas:
                canciones_recomendadas.add(cancion['Nombre'])
                canciones_similares.append({
                    'artista': artista_recomendado,
                    'genero': row['Género'],
                    'cancion_similar': cancion['Nombre'],
                    'similitud': cancion['similitud']
                })
                break  # Seleccionamos solo una canción similar por artista

    return canciones_similares

# Función principal para interactuar con el usuario
def obtener_recomendacion_artista():
    # Solicitar al usuario el nombre del artista que le interesa
    while True:
        artista_seleccionado = input("Por favor, ingrese el nombre del artista que le interesa: ")

        # Verificar si el artista existe en el dataframe
        if artista_seleccionado in df2_letras['Artista'].values:
            break  # Salir del bucle si el artista está en la base de datos
        else:
            print("El artista no se encuentra en el sistema. Intente nuevamente.")  # Mensaje de error

    # Llamar a la función de recomendación para artistas similares
    artistas_recomendados = recomendar_artistas(artista_seleccionado, df2_letras, tipo_recomendacion='vader')

    # Llamar a la función para obtener canciones similares entre los artistas recomendados
    canciones_similares = obtener_canciones_similares(artista_seleccionado, artistas_recomendados, df2_letras)

    # Mostrar los resultados
    print("\nRecomendaciones de artistas y canciones similares:")
    for recomendacion in canciones_similares:
        print(f"\nArtista recomendado: {recomendacion['artista']} ({recomendacion['genero']})")
        print(f"Canción similar: {recomendacion['cancion_similar']}")
        print(f"Similitud: {recomendacion['similitud']:.4f}")

# Llamar a la función principal para empezar la interacción con el usuario
obtener_recomendacion_artista()



Por favor, ingrese el nombre del artista que le interesa: Sabrina Carpenter

Recomendaciones de artistas y canciones similares:

Artista recomendado: Lady Gaga (Pop)
Canción similar: Die With A Smile
Similitud: 1.0000

Artista recomendado: Kendrick Lamar (Hip Hop)
Canción similar: squabble up
Similitud: 0.9973

Artista recomendado: Kendrick Lamar (Hip Hop)
Canción similar: luther (with sza)
Similitud: 0.9989


<ipython-input-5-b4be7137c91b>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canciones['similitud'] = similitudes[0]
<ipython-input-5-b4be7137c91b>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canciones['similitud'] = similitudes[0]
<ipython-input-5-b4be7137c91b>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

#**Sistema de Recomendación Musical Basado en el Estado Emocional**

El sistema busca proporcionar una experiencia personalizada, recomendando música que pueda aliviar o mejorar el estado emocional del usuario, ya sea proporcionando canciones relajantes para calmar la ansiedad o canciones energéticas para levantar el ánimo en caso de depresión

- ¿Cómo funciona?
Selección de la condición emocional: El usuario ingresa su condición emocional actual, como "ansiedad", "insomnio", "toc" (trastorno obsesivo-compulsivo) o "depresión".

Filtrado de canciones: Dependiendo de la condición emocional seleccionada:

- Para condiciones como ansiedad, insomnio o TOC, el sistema recomienda canciones relajantes (con acústica alta, danceabilidad baja, energía baja, y géneros como música clásica, instrumental o lo-fi).
- Para depresión, recomienda canciones energéticas (con acústica baja, danceabilidad alta, energía alta, y géneros como pop, rock o k-pop).
- Recomendación personalizada: El sistema calcula la similitud entre las canciones basándose en características musicales como la valencia (positividad) y la energía. Luego, muestra las tres canciones más recomendadas según la condición emocional del usuario.

Interacción continua: El usuario puede decidir si quiere más canciones recomendadas. Si responde "sí", el sistema muestra otras tres canciones similares, y así sucesivamente hasta que el usuario indique que ya no desea más canciones.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Función para recomendar canciones según la condición emocional del usuario
def recomendar_canciones_por_emocion(condicion_emocional, df):
    # Aplicar los filtros basados en las condiciones emocionales
    if condicion_emocional in ['ansiedad', 'insomnio', 'toc']:
        # Filtro para canciones relajantes (acústica alta, danceabilidad baja, energía baja)
        df_filtrado = df2_letras[
             (df2_letras['Género'].isin(['Classical & Orchestral', 'Instrumental & Relaxation', 'Video Game Music', 'Indie', 'R&B/Soul', 'Lo-fi & Chill', 'Folk']))
        ]
    elif condicion_emocional == 'depresion':
        # Filtro para canciones energéticas (acústica baja, danceabilidad alta, energía alta)
        df_filtrado = df2_letras[
            (df2_letras['Género'].isin(['K-Pop', 'Rock', 'Country', 'Pop']))
        ]
    else:
        return None  # Si no es una condición válida

    # Verificar si el dataframe tiene suficientes canciones después del filtrado
    if df_filtrado.empty:
        return None  # Si no hay canciones, devolvemos None

    # Calcular la similitud entre las canciones basándonos en características como Valence y Energy
    df_filtrado['similitud'] = cosine_similarity(df_filtrado[['Valence', 'Energy']], df_filtrado[['Valence', 'Energy']]).mean(axis=1)

    # Ordenar las canciones por similitud
    df_filtrado = df_filtrado.sort_values('similitud', ascending=False)

    # Seleccionar las tres canciones más recomendadas
    recomendacion = df_filtrado.head(3)

    return recomendacion[['Nombre', 'Artista', 'Género', 'similitud']]

# Función principal para interactuar con el usuario
def obtener_recomendacion_emocion():
    # Solicitar al usuario la condición emocional
    while True:
        condicion_usuario = input("Por favor, ingrese su condición emocional (ansiedad, insomnio, toc, depresion): ").lower()
        if condicion_usuario in ['ansiedad', 'insomnio', 'toc', 'depresion']:
            break
        else:
            print("Entrada no válida. Por favor ingrese una condición emocional válida.")

    while True:  # Bucle para continuar recomendando canciones hasta que el usuario diga que no
        # Llamar a la función de recomendación para obtener las canciones
        resultado = recomendar_canciones_por_emocion(condicion_usuario, df2_letras)

        # Verificar si hay canciones recomendadas
        if resultado is None:
            print("\nLo siento, no encontramos canciones que coincidan con su condición emocional.")
            break

        # Mostrar las recomendaciones
        print("\nCanciones recomendadas basadas en su condición emocional:")
        for idx, row in resultado.iterrows():
            print(f"\nCanción recomendada: {row['Nombre']}")
            print(f"Artista: {row['Artista']}")
            print(f"Género: {row['Género']}")
            print(f"Similitud: {row['similitud']:.4f}")

        # Preguntar si el usuario necesita más canciones
        while True:
            respuesta = input("\n¿Le gustaría más canciones recomendadas? (si/no): ").lower()
            if respuesta == 'si':
                break  # Si el usuario quiere más canciones, se repite el ciclo
            elif respuesta == 'no':
                print("\nGracias por usar el sistema de recomendación.")
                return  # Si el usuario no quiere más canciones, terminamos la función
            else:
                print("Respuesta no válida. Por favor ingrese 'sí' o 'no'.")

# Llamar a la función principal para empezar la interacción con el usuario
obtener_recomendacion_emocion()


Por favor, ingrese su condición emocional (ansiedad, insomnio, toc, depresion): toc


<ipython-input-6-8c2aac2e069b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['similitud'] = cosine_similarity(df_filtrado[['Valence', 'Energy']], df_filtrado[['Valence', 'Energy']]).mean(axis=1)



Canciones recomendadas basadas en su condición emocional:

Canción recomendada: What's The Difference
Artista: Dr. Dre
Género: R&B/Soul
Similitud: 0.9651

Canción recomendada: Flowers In Your Hair
Artista: The Lumineers
Género: Folk
Similitud: 0.9651

Canción recomendada: Talkin' Bout a Revolution
Artista: Tracy Chapman
Género: Folk
Similitud: 0.9651

¿Le gustaría más canciones recomendadas? (si/no): no

Gracias por usar el sistema de recomendación.
